In [5]:
import tensorflow as tf
import numpy as np
import random
from collections import deque
import osmnx as ox
import networkx as nx
import folium
from matplotlib import cm
import matplotlib.colors as mcolors
import osmnx as ox
from geopy.geocoders import Nominatim
import time
import osmnx as ox
import networkx as nx
import pandas as pd
import os
from geopy.geocoders import Nominatim
import pickle

In [10]:
# STEP 1: MENGAMBIL DATA JALAN dari OSM

# Daftar kota yang akan diproses
cities = ["Surabaya, Indonesia", "Malang, Indonesia", "Kediri, Indonesia", "Sidoarjo, Indonesia", "Gresik, Indonesia"]
data_dir = "../data"  # lokasi penyimpanan file .pkl

for city_name in cities:
    safe_name = city_name.split(",")[0].replace(" ", "_").lower()
    filepath = f"{data_dir}/road_graph_{safe_name}.pkl"
    
    if os.path.exists(filepath):
        print(f"✅ Data jalan {city_name} sudah tersedia.")
    else:
        print(f"📥 Mengambil data jalan dari OSM: {city_name}...")
        
        # Ambil jaringan jalan yang bisa dilalui mobil
        G = ox.graph_from_place(city_name, network_type="drive", simplify=True)
        
        # Simpan ke file .pkl
        nx.write_gpickle(G, filepath)
        print(f"💾 Data jalan {city_name} disimpan sebagai {filepath}")

✅ Data jalan Surabaya, Indonesia sudah tersedia.
✅ Data jalan Malang, Indonesia sudah tersedia.
✅ Data jalan Kediri, Indonesia sudah tersedia.
✅ Data jalan Sidoarjo, Indonesia sudah tersedia.
✅ Data jalan Gresik, Indonesia sudah tersedia.


In [11]:
# STEP 2: MEMUAT DATA SPKLU dari CSV

spklu_path = f"{data_dir}/spklu_dataset.csv"
spklu_df = pd.read_csv(spklu_path)

print("📊 Contoh data SPKLU:")
print(f"📄 Jumlah SPKLU dalam dataset: {len(spklu_df)}")

display(spklu_df.head())

📊 Contoh data SPKLU:
📄 Jumlah SPKLU dalam dataset: 10


,Nama SPKLU,Alamat,Latitude,Longitude,Jenis Konektor & Daya
0,SPKLU Gedung Pemkot Surabaya,"PPRX+R52 Ketabang, Surabaya, East Java",-7.257996,112.747887,AC TYPE 2 - 22KW
1,SPKLU Balai Kota Surabaya,"PPRX+R52 Ketabang, Surabaya, East Java",-7.257996,112.747887,AC TYPE 2 - 22KW
2,SPKLU WTC Surabaya Mall,"PPPX+X7C Embong Kaliasin, Surabaya, East Java",-7.262561,112.748145,"AC TYPE - 22KW, CCS 2 - 60KW DC, CHAdeMO - 60K..."
3,SPKLU PLN UID Jawa Timur,"PPMV+P77 Embong Kaliasin, Surabaya, East Java",-7.265702,112.743163,"AC TYPE - 7KW, CCS 2 - 100KW DC"
4,SPKLU Emergency Mobile,"PPMV+J63 Embong Kaliasin, Surabaya, East Java",-7.265988,112.743054,"AC TYPE - 22KW, CCS 2 - 30KW DC"


In [12]:
import pandas as pd

# Fungsi untuk membaca data SPKLU
def load_spklu_data(file_path):
    try:
        spklu_data = pd.read_csv(file_path)
        print(f"SPKLU Data Loaded: {len(spklu_data)} stations available.")
        return spklu_data
    except Exception as e:
        print(f"Error occurred while loading SPKLU data: {e}")
        return None

# Fungsi untuk mencari SPKLU terdekat berdasarkan lokasi dan jenis konektor
def find_nearest_spklu(current_location, selected_connector, spklu_data):
    # Filter SPKLU berdasarkan jenis konektor
    spklu_filtered = spklu_data[spklu_data['connector_type'] == selected_connector]
    
    # Menghitung jarak antara lokasi pengguna dan SPKLU (misal, menggunakan Haversine)
    nearest_spklu = None
    min_distance = float('inf')
    
    for _, spklu in spklu_filtered.iterrows():
        spklu_location = (spklu['latitude'], spklu['longitude'])
        distance = calculate_distance(current_location, spklu_location)
        if distance < min_distance:
            min_distance = distance
            nearest_spklu = spklu
    
    return nearest_spklu

# Fungsi untuk menghitung jarak antar dua lokasi (Haversine formula)
from math import radians, sin, cos, sqrt, atan2

def calculate_distance(loc1, loc2):
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    
    # Konversi derajat ke radian
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Rumus Haversine
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius = 6371  # Radius bumi dalam km
    
    distance = radius * c  # Jarak dalam km
    return distance

# Fungsi utama untuk menjalankan proses
def main():
    # Load SPKLU dataset
    spklu_file = "spklu_dataset.csv"  # Path ke dataset SPKLU Anda
    spklu_data = load_spklu_data(spklu_file)
    
    if not spklu_data:
        return
    
    # Ambil lokasi depot dan pelanggan serta data kendaraan dan konektor
    locations, depot_location = get_locations_from_input()
    selected_battery, selected_connector = choose_battery_and_connector()
    
    # Menampilkan lokasi-lokasi yang dipilih
    print("\nLokasi awal: Depot", depot_location)
    for idx, location in enumerate(locations[1:], start=1):
        print(f"Lokasi tujuan {idx}: {location}")
    
    # Menemukan SPKLU terdekat dari lokasi awal (Depot)
    nearest_spklu = find_nearest_spklu(depot_location, selected_connector, spklu_data)
    if nearest_spklu is not None:
        print(f"\nSPKLU terdekat dari Depot: {nearest_spklu['station_name']} - {nearest_spklu['latitude']}, {nearest_spklu['longitude']}")
    else:
        print("Tidak ada SPKLU yang sesuai dengan jenis konektor di dekat Depot.")
    
    # Output data kendaraan dan konektor
    print(f"\nKapasitas Baterai Mobil: {selected_battery['model']}")
    print(f"Jenis Konektor: {selected_connector}")

# Menjalankan proses utama
main()


Error occurred while loading SPKLU data: [Errno 2] No such file or directory: 'spklu_dataset.csv'
